<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/MobileNetV2_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MobileNetv2 feature extraction transfer learning

### **mission** :we want to classify the images of ten different foods, using the MobileNetv2 and transfer learning, using only 10% of the training data which is only 75 photos in each class, the test set contains 250 images in each class. 

In [19]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
from tensorflow.keras import layers, optimizers
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

##load data:

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-02-28 08:14:51--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.74.128, 209.85.145.128, 209.85.146.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.74.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   208MB/s    in 0.8s    

2022-02-28 08:14:52 (208 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [4]:
for dirpath, dirnames,filenames in os.walk('10_food_classes_10_percent'):
  print(f'there are {len(dirnames)} folders and {len(filenames)} images in {dirpath}')

there are 2 folders and 0 images in 10_food_classes_10_percent
there are 10 folders and 0 images in 10_food_classes_10_percent/test
there are 0 folders and 250 images in 10_food_classes_10_percent/test/fried_rice
there are 0 folders and 250 images in 10_food_classes_10_percent/test/chicken_wings
there are 0 folders and 250 images in 10_food_classes_10_percent/test/pizza
there are 0 folders and 250 images in 10_food_classes_10_percent/test/steak
there are 0 folders and 250 images in 10_food_classes_10_percent/test/grilled_salmon
there are 0 folders and 250 images in 10_food_classes_10_percent/test/sushi
there are 0 folders and 250 images in 10_food_classes_10_percent/test/chicken_curry
there are 0 folders and 250 images in 10_food_classes_10_percent/test/ramen
there are 0 folders and 250 images in 10_food_classes_10_percent/test/hamburger
there are 0 folders and 250 images in 10_food_classes_10_percent/test/ice_cream
there are 10 folders and 0 images in 10_food_classes_10_percent/train


## Preprocess data:

In [5]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

In [6]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [7]:
train_data = train_datagen.flow_from_directory(train_dir,target_size=(224,224))
test_data = test_datagen.flow_from_directory(test_dir,target_size=(224,224))

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


## building model 

In [8]:
MobileNetv2 = 'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5'
feature_Extractor_layers = hub.KerasLayer(MobileNetv2,name='MobileNetv2',
                                          trainable=False,input_shape= (224,224,3))

In [36]:
MobileNetmodel = tf.keras.Sequential([
                             
              feature_Extractor_layers,

              # layers.Dense(128,activation='relu'),
              layers.Dense(10,activation='softmax')
])

MobileNetmodel.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
                       optimizer = tf.keras.optimizers.Adam(),
                       metrics = ['accuracy'])

In [37]:
MobileNetmodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobileNetv2 (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 10)                12810     
                                                                 
Total params: 2,270,794
Trainable params: 12,810
Non-trainable params: 2,257,984
_________________________________________________________________


## creat tensorboard callbacks:

In [38]:
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + '/' +datetime.datetime.now().strftime('%y%m%d-%H%M%S')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_images=True)
  print(f'saving tensorboard log files to: {log_dir}')
  return tensorboard_callback


## fit the model with 10% of the training data:

In [39]:
MobileNetmodel.fit(train_data, epochs=15,
                   validation_data = test_data, 
                   callbacks=create_tensorboard_callback('tensorboard', 'MobileNetv2'))

saving tensorboard log files to: tensorboard/MobileNetv2/220228-082950
Epoch 1/15
24/24 [==============================] - 21s 838ms/step - loss: 1.8970 - accuracy: 0.3573 - val_loss: 1.2645 - val_accuracy: 0.6180
Epoch 2/15
24/24 [==============================] - 19s 806ms/step - loss: 1.0149 - accuracy: 0.7107 - val_loss: 0.8709 - val_accuracy: 0.7396
Epoch 3/15
24/24 [==============================] - 19s 804ms/step - loss: 0.7165 - accuracy: 0.8147 - val_loss: 0.7477 - val_accuracy: 0.7720
Epoch 4/15
24/24 [==============================] - 19s 797ms/step - loss: 0.5567 - accuracy: 0.8653 - val_loss: 0.6754 - val_accuracy: 0.7904
Epoch 5/15
24/24 [==============================] - 18s 790ms/step - loss: 0.4640 - accuracy: 0.9027 - val_loss: 0.6417 - val_accuracy: 0.8020
Epoch 6/15
24/24 [==============================] - 25s 1s/step - loss: 0.3928 - accuracy: 0.9173 - val_loss: 0.6126 - val_accuracy: 0.8096
Epoch 7/15
24/24 [==============================] - 18s 784ms/step - loss:

In [40]:
MobileNetmodel.evaluate(test_data)

79/79 [==============================] - 15s 183ms/step - loss: 0.5409 - accuracy: 0.8256


[0.5409265756607056, 0.8256000280380249]

In [41]:
y_pred = np.argmax(MobileNetmodel.predict(test_data),axis=1)
y_pred

array([0, 9, 5, ..., 9, 0, 0])

## upload data to tensorboard:


In [42]:
!tensorboard dev upload --logdir ./tensorboard/ \
--name 'MobileNetv2 results on food data' \
--one_shot


New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/AUNsYb2eRv2kD78QQmPCdA/

[2022-02-28T08:35:15] Started scanning logdir.
[2022-02-28T08:35:18] Total uploaded: 128 scalars, 0 tensors, 3 binary objects (4.6 MB)
[2022-02-28T08:35:18] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/AUNsYb2eRv2kD78QQmPCdA/


In [43]:
!tensorboard dev list

https://tensorboard.dev/experiment/AUNsYb2eRv2kD78QQmPCdA/
	Name                 MobileNetv2 results on food data
	Description          [No Description]
	Id                   AUNsYb2eRv2kD78QQmPCdA
	Created              2022-02-28 08:35:15 (14 seconds ago)
	Updated              2022-02-28 08:35:18 (11 seconds ago)
	Runs                 5
	Tags                 5
	Scalars              128
	Tensor bytes         0
	Binary object bytes  4864363
https://tensorboard.dev/experiment/umJfYK9XQjaQF6VKBQG3rw/
	Name                 MobileNetv2 results on food data
	Description          [No Description]
	Id                   umJfYK9XQjaQF6VKBQG3rw
	Created              2022-02-28 08:26:04 (9 minutes ago)
	Updated              2022-02-28 08:26:04 (9 minutes ago)
	Runs                 1
	Tags                 3
	Scalars              20
	Tensor bytes         0
	Binary object bytes  1620613
https://tensorboard.dev/experiment/YQvKfa5ZS66UqBWy8nMHgQ/
	Name                 efffient net vs. resnet
	Descripti